In [222]:
import numpy as np
import itertools as it 
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor

# Guide and Ideas for Capstone Project

We begin this guide by downloading the data:

In [223]:
file = 8

In [224]:
def load_data(dim):
    X = np.load('data/initial_data/function_' + str(dim) +'/initial_inputs.npy')
    y = np.load('data/initial_data/function_' + str(dim) +'/initial_outputs.npy')
    return X,y

X, y = load_data(file)
dimension = np.shape(X)[1] 
print("X=", X.shape, ", dimensions=", dimension)
for i in range(dimension):
    print("x", str(i+1), "=", X[:,i][:4], "... ")
print("Y=", y.shape, "=", y[:4], "...")

X= (40, 8) , dimensions= 8
x 1 = [0.60499445 0.17800696 0.00907698 0.50602816] ... 
x 2 = [0.29221502 0.56622265 0.81162615 0.65373012] ... 
x 3 = [0.90845275 0.99486184 0.52052036 0.36341078] ... 
x 4 = [0.35550624 0.21032501 0.07568668 0.17798105] ... 
x 5 = [0.20166872 0.32015266 0.26511183 0.0937283 ] ... 
x 6 = [0.57533801 0.70790879 0.09165169 0.19742533] ... 
x 7 = [0.31031095 0.63538449 0.59241515 0.7558269 ] ... 
x 8 = [0.73428138 0.10713163 0.36732026 0.29247234] ... 
Y= (40,) = [7.3987211  7.00522736 8.45948162 8.28400781] ...


In [225]:
if (dimension == 2):
    x1 =X[:,0]
    x2 =X[:,1] 
    plt.scatter(x1, x2, c=y)
    for (i, j, text) in zip(x1, x2, y):
        plt.annotate(format(text, ".2e"),  # This is the text to use for the annotation
                 (i, j),  # This is the point to label
                 textcoords="offset points",  # how to position the text
                 xytext=(0,10),  # distance from text to points (x,y)
                 ha='right')  # horizontal alignment can be left, right or center
    plt.show()

# Upper Confidence Bound

A second alternative would be to use Bayesian Optimization and consider an Upper Confidence Bound acquisition function: 

In [226]:
# Add here hyper-parameters
gpr = GaussianProcessRegressor()
gpr.fit(X, y)



GaussianProcessRegressor()

In [227]:
def find_maxiumum(X, y):
    maximum_of_X = []
    maximum_so_far = -1e99
    for index in range(len(y)):
        if (y[index] > maximum_so_far):
            maximum_so_far = y[index]
            maximum_of_X = []
            for i in range(dimension):
                maximum_of_X.append(X[index, i])
    return np.array(maximum_of_X), maximum_so_far
    
#X_sample =np.array([[1,10], [2,20], [3,30], [4, 40]])
#Xmax, ymax = find_maxiumum(X_sample, [3,2,5,1])
Xmax, ymax = find_maxiumum(X, y)
XmaxLower = Xmax - (Xmax * 0.2)
XmaxUpper = Xmax + (Xmax * 0.2)
print("maximum X=", Xmax, ", maximum y=", ymax)

sizes = [ [2, 100],
          [3,  80],
          [4,  60],
          [5,  30],
          [6,  15],
          [7,  10],
          [8,  8]]

#Generate each dimension 
size = sizes[dimension - 2][1]

data = []
for item in range(dimension):
    data.append(np.linspace(max(XmaxLower[item], 0 + 1e-10), min(XmaxUpper[item], 1 - 1e-10), size))

x_grid_points = np.array(data)

print("data-points=", x_grid_points.shape)
for dim in range(dimension):
    print("data-points[", dim + 1, "]=", x_grid_points[dim][:5], "...", x_grid_points[dim][-1] )



maximum X= [0.05644741 0.06595555 0.02292868 0.03878647 0.40393544 0.80105533
 0.48830701 0.89308498] , maximum y= 9.598482002566342
data-points= (8, 8)
data-points[ 1 ]= [0.04515793 0.0483835  0.05160906 0.05483463 0.05806019] ... 0.06773689327873403
data-points[ 2 ]= [0.05276444 0.05653333 0.06030222 0.06407111 0.06784   ] ... 0.07914666413125185
data-points[ 3 ]= [0.01834294 0.01965315 0.02096336 0.02227357 0.02358378] ... 0.027514413587452945
data-points[ 4 ]= [0.03102918 0.03324555 0.03546192 0.03767829 0.03989466] ... 0.04654376683093338
data-points[ 5 ]= [0.32314835 0.34623038 0.3693124  0.39239443 0.41547645] ... 0.48472252862253273
data-points[ 6 ]= [0.64084426 0.68661885 0.73239344 0.77816803 0.82394262] ... 0.9612663950666926
data-points[ 7 ]= [0.39064561 0.41854886 0.44645212 0.47435538 0.50225864] ... 0.5859684082955923
data-points[ 8 ]= [0.71446798 0.75525827 0.79604856 0.83683885 0.87762913] ... 0.9999999999


In [228]:
X_grid = np.fromiter(it.chain(*it.product(*x_grid_points)), dtype=float).reshape(-1,dimension)
print("X_grid=", X_grid.shape, " = ", X_grid[0:5,:], "...", X_grid[-1,:])


X_grid= (16777216, 8)  =  [[0.04515793 0.05276444 0.01834294 0.03102918 0.32314835 0.64084426
  0.39064561 0.71446798]
 [0.04515793 0.05276444 0.01834294 0.03102918 0.32314835 0.64084426
  0.39064561 0.75525827]
 [0.04515793 0.05276444 0.01834294 0.03102918 0.32314835 0.64084426
  0.39064561 0.79604856]
 [0.04515793 0.05276444 0.01834294 0.03102918 0.32314835 0.64084426
  0.39064561 0.83683885]
 [0.04515793 0.05276444 0.01834294 0.03102918 0.32314835 0.64084426
  0.39064561 0.87762913]] ... [0.06773689 0.07914666 0.02751441 0.04654377 0.48472253 0.9612664
 0.58596841 1.        ]


In [229]:
#X_grid = np.array(X_grid)
mean, std = gpr.predict(X_grid, return_std = True)
ucb = mean + 1.96 * std

In [230]:
print("UCB=", ucb.shape, ", value=", ucb[:5])
print("mean=", mean.shape, ", value=", mean[:5])
#print("mean + std(1.96)=", mean.shape, ", value=", (mean + std * 1.96)[:5])


UCB= (16777216,) , value= [10.24445569 10.16791958 10.08598454 10.00089471  9.91492033]
mean= (16777216,) , value= [9.91103731 9.86048108 9.7997926  9.72911235 9.64860456]


In [231]:
idx_max = np.argmax(ucb)
next_query = X_grid[idx_max]
print(next_query)
best_numbers = []
for item in next_query:
    best_numbers.append(min(max(item, 0.000001), 0.999999))
print(best_numbers)


[0.06773689 0.07914666 0.02751441 0.04654377 0.4616405  0.64084426
 0.39064561 0.71446798]
[0.06773689327873403, 0.07914666413125185, 0.027514413587452945, 0.04654376683093338, 0.46164050345003116, 0.6408442633777951, 0.39064560553039485, 0.7144679812318023]


Doing this every week, we expect to be able to find the first of the two modes! This should would be enough to provide a good solution to the problem. To really verify if we found the maximum, we would need to find a second mode, however, we may be unlucky and simply never find it. This is an important part of Machine Learning: in theory we might want optimal, perfect solutions, but in practice most of the advancements can be done with simple solutions and careful consideration of the data available to us.

In [232]:
np.array2string(np.array(best_numbers), precision=6, separator='-', floatmode='fixed', formatter={'float': '{:0.6f}'.format})

'[0.067737-0.079147-0.027514-0.046544-0.461641-0.640844-0.390646-0.714468]'